In [23]:
from pathlib import Path

import pandas as pd

from utils import check_date_sequence_integrity

from osgeo import gdal
import numpy as np
import xarray as xr

In [24]:
SLM10_dir = Path(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\run\pkg_SLM10\data\output')
SLM10_filename_list = [f.name for f in SLM10_dir.glob('*.flt')]
check_date_sequence_integrity(SLM10_filename_list)
SLM10_filelist = list(SLM10_dir.glob('*.flt'))

文件日期序列是连续的.
起始日期: 2018-01-01
结束日期: 2020-12-31


In [25]:
SLM20_dir = Path(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\run\pkg_SLM20\data\output')
SLM20_filename_list = [f.name for f in SLM20_dir.glob('*.flt')]
check_date_sequence_integrity(SLM20_filename_list)
SLM20_filelist = list(SLM20_dir.glob('*.flt'))

文件日期序列是连续的.
起始日期: 2018-01-01
结束日期: 2020-12-31


In [26]:
SLM40_dir = Path(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\run\pkg_SLM40\data\output')
SLM40_filename_list = [f.name for f in SLM40_dir.glob('*.flt')]
check_date_sequence_integrity(SLM40_filename_list)
SLM40_filelist = list(SLM40_dir.glob('*.flt'))

文件日期序列是连续的.
起始日期: 2018-01-01
结束日期: 2020-12-31


In [27]:
AVG_dir = Path(r'D:\PROJECTS\hnsw_soil_moisture_surface\auto_anu\pkg_run3\data\output')
AVG_filename_list = [f.name for f in AVG_dir.glob('*.flt')]
check_date_sequence_integrity(AVG_filename_list)
AVG_filelist = list(AVG_dir.glob('*.flt'))

文件日期序列是连续的.
起始日期: 2018-01-01
结束日期: 2020-12-31


In [28]:
# 读取第一个文件，获取经纬度信息

first_flt = AVG_filelist[0]
dst1 = gdal.Open(str(first_flt))
data1 = dst1.GetRasterBand(1).ReadAsArray()
geotransform = dst1.GetGeoTransform()

lon = np.arange(geotransform[0], geotransform[0] + geotransform[1] * data1.shape[1], geotransform[1])
lat = np.arange(geotransform[3], geotransform[3] + geotransform[5] * data1.shape[0], geotransform[5])

In [29]:
# 读取所有文件
SLM10_data = np.zeros((len(AVG_filelist), data1.shape[0], data1.shape[1]), dtype=np.float32)
# 循环读入文件放入data数组
for i, f in enumerate(SLM10_filelist):
    dst = gdal.Open(str(f))
    SLM10_data[i] = dst.GetRasterBand(1).ReadAsArray()
    dst = None

SLM20_data = np.zeros((len(AVG_filelist), data1.shape[0], data1.shape[1]), dtype=np.float32)
for i, f in enumerate(SLM20_filelist):
    dst = gdal.Open(str(f))
    SLM20_data[i] = dst.GetRasterBand(1).ReadAsArray()
    dst = None

SLM40_data = np.zeros((len(AVG_filelist), data1.shape[0], data1.shape[1]), dtype=np.float32)
for i, f in enumerate(SLM40_filelist):
    dst = gdal.Open(str(f))
    SLM40_data[i] = dst.GetRasterBand(1).ReadAsArray()
    dst = None
    
AVG_data = np.zeros((len(AVG_filelist), data1.shape[0], data1.shape[1]), dtype=np.float32)
for i, f in enumerate(AVG_filelist):
    dst = gdal.Open(str(f))
    AVG_data[i] = dst.GetRasterBand(1).ReadAsArray()
    dst = None


In [30]:
SLM10_data, SLM20_data, SLM40_data, AVG_data

(array([[[25.222786 , 24.792393 , 26.447512 , ..., 19.830473 ,
          19.24925  , 18.996387 ],
         [24.256176 , 23.817545 , 24.550974 , ..., 19.44513  ,
          19.280247 , 19.50865  ],
         [23.78947  , 25.460043 , 24.02026  , ..., 20.364876 ,
          19.197054 , 18.932734 ],
         ...,
         [20.103626 , 20.060606 , 19.894325 , ..., 20.281124 ,
          20.111616 , 19.201403 ],
         [20.10731  , 20.06019  , 19.8788   , ..., 19.28627  ,
          18.788473 , 18.427343 ],
         [19.986954 , 19.849285 , 19.820415 , ..., 18.939188 ,
          18.30287  , 18.720005 ]],
 
        [[25.444931 , 24.993435 , 26.337759 , ..., 19.803434 ,
          19.245844 , 18.96777  ],
         [24.574507 , 24.111914 , 24.654127 , ..., 19.466522 ,
          19.26279  , 19.39413  ],
         [24.129196 , 25.460554 , 24.12986  , ..., 20.232782 ,
          19.179455 , 18.892897 ],
         ...,
         [21.448212 , 21.329927 , 21.101322 , ..., 20.079811 ,
          19.881914 , 19

In [31]:
# 生成时间序列
date_range = pd.date_range(start='2018-01-01', periods=len(file_list), freq='D')

In [32]:
# 输出到NC文件
ds = xr.Dataset(
    {
        'SLM10': (['time', 'latitude', 'longitude'], SLM10_data),
        'SLM20': (['time', 'latitude', 'longitude'], SLM20_data),
        'SLM40': (['time', 'latitude', 'longitude'], SLM40_data),
        'AVG': (['time', 'latitude', 'longitude'], AVG_data)

    },
    coords={
        'time': date_range,
        'latitude': lat,
        'longitude': lon
    }
)

ds.attrs['title'] = 'Soil Moisture Content'
ds.attrs['description'] = 'This is a Soil Moisture Surface'

ds['SLM10'].attrs['units'] = '%'
ds['SLM10'].attrs['long_name'] = '10cm Soil Moisture Content'

ds['SLM20'].attrs['units'] = '%'
ds['SLM20'].attrs['long_name'] = '20cm Soil Moisture Content'

ds['SLM40'].attrs['units'] = '%'
ds['SLM40'].attrs['long_name'] = '40cm Soil Moisture Content'

ds['AVG'].attrs['units'] = '%'
ds['AVG'].attrs['long_name'] = 'Average Soil Moisture Content'



ds

<xarray.Dataset>
Dimensions:    (time: 1096, latitude: 24, longitude: 24)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01 2018-01-02 ... 2020-12-31
  * latitude   (latitude) float64 29.88 29.62 29.38 29.12 ... 24.62 24.38 24.12
  * longitude  (longitude) float64 109.1 109.4 109.6 109.9 ... 114.4 114.6 114.9
Data variables:
    SLM10      (time, latitude, longitude) float32 25.22 24.79 ... 10.42 10.39
    SLM20      (time, latitude, longitude) float32 23.82 23.74 ... 17.81 19.25
    SLM40      (time, latitude, longitude) float32 26.07 25.81 ... 15.64 14.22
    AVG        (time, latitude, longitude) float32 25.12 24.86 ... 17.72 18.08
Attributes:
    title:        Soil Moisture Content
    description:  This is a Soil Moisture Surface

In [33]:
ds.to_netcdf(r'D:\PROJECTS\hnsw_soil_moisture_surface\soil_moisture_all.nc')